In [1]:
import pandas as pd
import numpy as np
import datetime
import math

In [2]:
def sign(z):
    if z > 0:
        return 1
    else:
        return -1

In [3]:
# 算epsilon_t用
def count_weighted_epsilon(weights, real_y, predicted_y):  # 皆為 numpy
    error = 0
    for i in range(len(real_y)):
        if real_y[i] != predicted_y[i]:
            error += weights[i]
    epsilon = error / sum(weights)
    return epsilon

In [4]:
# 找decision stump的Ein_u用
def count_e_with_u(weights, real_y, predicted_y):  # 皆為 numpy
    error = 0
    for i in range(len(real_y)):
        if real_y[i] != predicted_y[i]:
            error += weights[i]
    e_u = error / len(real_y)
    return e_u

In [5]:
# 算Ein, Eout用
def count_e(real_y, predicted_y):  # 皆為 numpy
    error = 0
    for i in range(len(real_y)):
        if real_y[i] != predicted_y[i]:
            error += 1
    e = error / len(real_y)
    return e

In [6]:
def decision_stump(weights, data_x, data_y):  # 皆為 numpy
    min_Ein_u = float('inf')
    best_s = 's'
    best_theta = 't'
    best_feature = 'i'
    
    s_list = [1, -1]
    
    for i in range(len(data_x[0])):  # 跑每個feature
        min_Ein_u_i = float('inf')
        best_s_i = 's'
        best_theta_i = 't'

        x_i = [data_x[j][i] for j in range(len(data_x))]  # 每筆data第i個feature
        x_i.sort()  # 排序由小排到大
        
        # 先算theta = -inf的情況
        now_theta = float('-inf')
        for s in s_list:
            predicted_y = np.array([s * sign(data_x[j][i] - now_theta) for j in range(len(data_x))])  # h(x)
            now_epsilon = count_e_with_u(weights, data_y, predicted_y)  # 算Ein_u
            
            if now_epsilon < min_Ein_u_i:  # 比此feature中最小的還好
                min_Ein_u_i = now_epsilon
                best_s_i = s
                best_theta_i = now_theta
        
        # theta = 任2數據中點的情況
        for n in range(len(x_i) - 1):
            now_theta = (x_i[n] + x_i[n + 1]) / 2
            for s in s_list:
                predicted_y = [s * sign(data_x[j][i] - now_theta) for j in range(len(data_x))]  # h(x)
                now_epsilon = count_e_with_u(weights, data_y, predicted_y)
                
                if now_epsilon < min_Ein_u_i:  # 比此feature中最小的還好
                    min_Ein_u_i = now_epsilon
                    best_s_i = s
                    best_theta_i = now_theta
        
        if min_Ein_u_i < min_Ein_u:  # Ein_u_i比這輪中最小的還好
            min_Ein_u = min_Ein_u_i
            best_s = best_s_i
            best_theta = best_theta_i
            best_feature = i
    return best_s, best_theta, best_feature

In [7]:
training_data = pd.read_excel('hw6_train.xlsx')
testing_data = pd.read_excel('hw6_test.xlsx')
print('training_size:', len(training_data))
print('testing_size:', len(testing_data))

training_size: 1000
testing_size: 1000


In [8]:
train_X = training_data.drop('y', axis=1)
train_X_np = np.asarray(train_X)
train_y = training_data['y']
train_y_np = np.array(train_y)

In [9]:
test_X = testing_data.drop('y', axis=1)
test_X_np = np.asarray(test_X)
test_y = testing_data['y']
test_y_np = np.array(test_y)

In [10]:
weight_u = np.array([1 / len(train_X) for i in range(len(train_X))])  # 初始weight
max_Ein_g = float('-inf')  # for Q12, Ein最大的g_t之Ein
find_first_005T = 0  # for Q13, 是否找到第一個T使Ein(G_T) <= 0.05, 0為否, 1為是

train_G_x_sum = np.array([0.] * len(train_y_np))  # 計算sigma(alpha_t * g_t(x_train))
test_G_x_sum = np.array([0.] * len(test_y_np))  # 計算sigma(alpha_t * g_t(x_test))
test_G_x_uniform_sum = np.array([0.] * len(test_y_np))  # 計算sigma(alpha_t * g_t(x_test)) for G_uniform

for t in range(500):
    if t % 10 == 0:
        print('starting iteration {}: {}'.format(t, datetime.datetime.now()))

    # 決定此輪decision stump
    s_t, theta_t, feature_t = decision_stump(weight_u, train_X_np, train_y_np)
    
    # 決定預測結果g_t(x)和epsilon, scaling和算Ein(G)用
    prediction_y_t = np.array([s_t * sign(train_X_np[j][feature_t] - theta_t) for j in range(len(train_X_np))])
    epsilon_t = count_weighted_epsilon(weight_u, train_y_np, prediction_y_t)
    predict_result = prediction_y_t * train_y_np  # 每個data point預測正確與否，正確為1，反之-1
    
    # for Q12, 決定Ein_g
    Ein_g = count_e(train_y_np, prediction_y_t)

    if Ein_g > max_Ein_g:  # Q12
        max_Ein_g = Ein_g
    
    if t == 0:  # Q11, Q14
        # 使用g1在testing data上做預測
        prediction_y_0_testd = np.array([s_t * sign(test_X_np[j][feature_t] - theta_t) for j in range(len(test_X_np))])
        Eout_g1 = count_e(test_y_np, prediction_y_0_testd)
        print('Ein_g1:', Ein_g)  # Q11
        print('Eout_g1:', Eout_g1)  # Q14
    
    # 更新scaling factor, weight_u, alpha_t
    scaling_factor = ((1-epsilon_t) / epsilon_t) ** 0.5
    weight_u = np.array([weight_u[j] * (scaling_factor ** -predict_result[j]) for j in range(len(weight_u))])
    alpha_t = math.log(scaling_factor)
    
    # 更新G(x)
    # for Q13, 在training data上執行G(x)更新，每輪 + alpha_t * g_t(x)
    if find_first_005T == 0:  # 只有還沒找到時需要持續更新此項目
        train_G_x_sum = np.array([train_G_x_sum[j] + (alpha_t * prediction_y_t[j]) for j in range(len(train_G_x_sum))])
        prediction_G_traind = np.array([sign(train_G_x_sum[j]) for j in range(len(train_G_x_sum))])  # 此輪G在training的預測結果
        Ein_G_T = count_e(train_y_np, prediction_G_traind)
        if Ein_G_T <= 0.05:
            find_first_005T = 1
            first_005T = t
    
    # for Q15, Q16, 在testing data上執行G(x)更新，每輪 + (alpha_t * ) g_t(x)
    prediction_y_t_testd = np.array([s_t * sign(test_X_np[j][feature_t] - theta_t) for j in range(len(test_X_np))])
    test_G_x_sum = np.array([test_G_x_sum[j] + (alpha_t * prediction_y_t_testd[j]) for j in range(len(test_G_x_sum))])
    test_G_x_uniform_sum = np.array([test_G_x_uniform_sum[j] + prediction_y_t_testd[j] for j in range(len(test_G_x_sum))])

starting iteration 0: 2022-01-07 23:46:49.946258
Ein_g1: 0.374
Eout_g1: 0.455
starting iteration 10: 2022-01-07 23:49:28.566514
starting iteration 20: 2022-01-07 23:52:03.929901
starting iteration 30: 2022-01-07 23:54:39.004326
starting iteration 40: 2022-01-07 23:57:23.600997
starting iteration 50: 2022-01-08 00:00:08.992310
starting iteration 60: 2022-01-08 00:02:45.789723
starting iteration 70: 2022-01-08 00:06:27.413173
starting iteration 80: 2022-01-08 00:08:59.703786
starting iteration 90: 2022-01-08 00:11:36.205059
starting iteration 100: 2022-01-08 00:14:09.999261
starting iteration 110: 2022-01-08 00:16:41.965642
starting iteration 120: 2022-01-08 00:19:17.545020
starting iteration 130: 2022-01-08 00:21:54.465549
starting iteration 140: 2022-01-08 00:24:27.747892
starting iteration 150: 2022-01-08 00:27:04.808799
starting iteration 160: 2022-01-08 00:29:41.174802
starting iteration 170: 2022-01-08 00:32:19.991349
starting iteration 180: 2022-01-08 00:34:54.076549
starting iter

In [11]:
# Q12
print('max Ein(g_t): {}'.format(max_Ein_g))

# Q13
print('smallest t s.t. Ein(G_T) <= 0.05: {}'.format(first_005T))

# Q15, Q16
prediction_G_testd = np.array([sign(test_G_x_sum[j]) for j in range(len(test_G_x_sum))])
Eout_G = count_e(test_y_np, prediction_G_testd)
print('Eout_G: {:.3f}'.format(Eout_G))

prediction_G_testd_uniform = np.array([sign(test_G_x_uniform_sum[j]) for j in range(len(test_G_x_uniform_sum))])
Eout_G_uniform = count_e(test_y_np, prediction_G_testd_uniform)
print('Eout_G_uniform: {:.3f}'.format(Eout_G_uniform))

max Ein(g_t): 0.591
smallest t s.t. Ein(G_T) <= 0.05: 354
Eout_G: 0.188
Eout_G_uniform: 0.225
